<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import os

In [ ]:
def main():
  #js_100 = pd.read_json("/content/drive/MyDrive/NLP/WLASL100.json")
  #folder = "/content/drive/MyDrive/NLP/dataset/subset_100/"
  js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
  folder = "/content/drive/MyDrive/NLP/dataset/"
  subset = False

  training_folder = folder + "train/"
  validation_folder = folder + "val/"
  test_folder = folder + "test/"

  training_video = training_folder + "video/"
  validation_video = validation_folder + "video/"
  test_video = test_folder + "video/"

  training_images = training_folder + "images/"
  validation_images = validation_folder + "images/"
  test_images = test_folder + "images/"

  os.makedirs(training_video, exist_ok=True)
  os.makedirs(validation_video, exist_ok=True)
  os.makedirs(test_video, exist_ok=True)

  os.makedirs(training_images, exist_ok=True)
  os.makedirs(validation_images, exist_ok=True)
  os.makedirs(test_images, exist_ok=True)

  # images with video_id < new_id belong to the original dataset (no augmentated data)
  new_id = 70380

  all_labels = [image.split("_")[0] for image in os.listdir(training_images)]
  set_labels = set(all_labels)
  # label_counts = {class : n. samples for class}
  label_counts = {label: all_labels.count(label) for label in set_labels}

  def add_gaussian_noise(seq, mean=0, std=0.01):
    noisy_seq = seq.copy()
    for i in range(0, 132, 4):
      noisy_seq[:, i:i+3] += np.random.normal(mean, std, (seq.shape[0], 3))
    for i in range(132, seq.shape[1], 3):
      noisy_seq[:, i:i+3] += np.random.normal(mean, std, (seq.shape[0], 3))

    return noisy_seq

  def jitter_joints(seq, jitter_scale=0.02):
    noisy_seq = seq.copy()
    for i in range(0, 132, 4):
      noisy_seq[:, i:i+3] += np.random.uniform(-jitter_scale, jitter_scale, (seq.shape[0], 3))
    for i in range(132, seq.shape[1], 3):
      noisy_seq[:, i:i+3] += np.random.uniform(-jitter_scale, jitter_scale, (seq.shape[0], 3))
    return noisy_seq

  def visibility_random_dropout(seq, threshold=0.1):
      seq_aug = seq.copy()

      for i in range(seq.shape[0]):
        for j in range(33):
          base_idx = j * 4
          visibility = seq_aug[i, base_idx + 3]

          # remove landmarks with low visibility and random dropout
          if visibility < threshold or np.random.rand() < threshold:
            seq_aug[i, base_idx:base_idx + 4] = 0.0

      return seq_aug

  def data_augmentation(images, max_count, new_id):
    for image in os.listdir(images):
      word, _ = image.split("_")
      if label_counts[word] < max_count:
        np_array = np.load(os.path.join(training_images, image))

        gauss_array = add_gaussian_noise(np_array)
        gauss_image = f"{word}_{new_id}.npy"
        new_id += 1
        np.save(os.path.join(training_images, gauss_image), gauss_array)

        jitter_points_array = jitter_joints(np_array)
        jitter_image = f"{word}_{new_id}.npy"
        new_id += 1
        np.save(os.path.join(training_images, jitter_image), jitter_points_array)

        drop_frames_array = visibility_random_dropout(np_array)
        drop_frames_image = f"{word}_{new_id}.npy"
        new_id += 1
        np.save(os.path.join(training_images, drop_frames_image), drop_frames_array)

  if subset:
    data_augmentation(training_images, 8, new_id)
  else:
    data_augmentation(training_images, 4, new_id)

if __name__ == "__main__":
  main()

WLASL100

original dataset:
- training set: 914
- validation set: 211
- test set: 189

after:
- training set: -
- validation set: 211
- test set: 189

WLASL2000

original dataset:
- training set: 8760
- validation set: 2493
- test set: 2060

after:
- training set: -
- validation set: 2493
- test set: 2060